In [1]:
import os
import glob
import pandas as pd
import ujson

In [2]:
clinical = pd.read_csv('OASIS3-clinical/OASIS3_ADRCclinicalData.csv')
scans = pd.read_csv('metadata/OASIS3_MR_json.csv')

In [3]:
first_T1s = []
for f in glob.glob("OASIS3-BIDS-first/*/*_sessions.tsv"):
    first_T1s.append(f'{os.path.basename(f).split("sub-")[-1].split("_sessions")[0]}_MR_'+pd.read_table(f, sep="\t").source_session_id[0])

In [4]:
scans = scans[scans.label.apply(lambda x: x in first_T1s)]
scans = scans[scans["scan category"] == "T1w"].sort_values(['label', 'filename'])
scans

,subject_id,label,acccession,release version,scan category,filename,Modality,MagneticFieldStrength,Manufacturer,ManufacturersModelName,...,PhaseOversampling,EchoNumber,Interpolation2D,RawImage,ConsistencyInfo,MultibandAccelerationFactor,PhaseEncodingAxis,AcquisitionDateTime,PhaseEncodingLines,AccelFactPE
16682,OAS30001,OAS30001_MR_d0129,CENTRAL_E09253,2018 Release,T1w,sub-OAS30001_ses-d0129_run-01_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14281,OAS30001,OAS30001_MR_d0129,CENTRAL_E09253,2018 Release,T1w,sub-OAS30001_ses-d0129_run-02_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30264,OAS30002,OAS30002_MR_d0371,CENTRAL_E11337,2018 Release,T1w,sub-OAS30002_ses-d0371_T1w.json,MR,1.5,Siemens,Sonata,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29434,OAS30003,OAS30003_MR_d0558,CENTRAL_E09260,2018 Release,T1w,sub-OAS30003_ses-d0558_run-01_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1520,OAS30003,OAS30003_MR_d0558,CENTRAL_E09260,2018 Release,T1w,sub-OAS30003_ses-d0558_run-02_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13950,OAS31169,OAS31169_MR_d0620,CENTRAL_E11124,2018 Release,T1w,sub-OAS31169_ses-d0620_run-02_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23062,OAS31170,OAS31170_MR_d2410,CENTRAL_E11125,2018 Release,T1w,sub-OAS31170_ses-d2410_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4380,OAS31171,OAS31171_MR_d0752,CENTRAL_E11126,2018 Release,T1w,sub-OAS31171_ses-d0752_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6936,OAS31172,OAS31172_MR_d0407,CENTRAL_E11127,2018 Release,T1w,sub-OAS31172_ses-d0407_run-01_T1w.json,MR,3.0,Siemens,TrioTim,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


=> Multiple runs per session, may be unclear to associate scans with this json

In [5]:
scans_sessions_df = []
for f in glob.glob("OASIS3-BIDS-first/*/*_sessions.tsv"):
    tbl = pd.read_table(f, sep='\t')
    tbl["subject_id"] = os.path.basename(f).split("sub-")[-1].split("_sessions")[0]
    first_visit = sorted(tbl.session_id)[0]
    tbl[tbl.session_id == first_visit]
    scans_sessions_df.append(tbl)

scans_sessions_df = pd.concat(scans_sessions_df)

In [6]:
scans_sessions_df

,session_id,source_session_id,age,mmse,cdr,commun,dx1,homehobb,judgment,memory,orient,perscare,sumbox,apoe,subject_id
0,ses-M006,d0129,65,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,0.0,0.0,1.5,33.0,OAS30001
0,ses-M012,d0371,68,28.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,23.0,OAS30002
0,ses-M018,d0558,60,29.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,33.0,OAS30003
0,ses-M036,d1101,58,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,23.0,OAS30004
0,ses-M006,d0143,48,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,0.0,0.0,1.5,33.0,OAS30005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ses-M006,d0148,65,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,0.0,0.0,1.5,33.0,OAS31168
0,ses-M018,d0620,64,29.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,33.0,OAS31169
0,ses-M078,d2410,71,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,33.0,OAS31170
0,ses-M024,d0752,68,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,0.0,0.0,0.0,23.0,OAS31171


In [7]:
scans_sessions_df.dx1.value_counts()

Cognitively normal           777
AD Dementia                  301
uncertain dementia             3
Non AD dem- Other primary      1
Unc: ques. Impairment          1
Name: dx1, dtype: int64

In [8]:
subjects_sessions = glob.glob("OASIS3-BIDS-first/sub-*/ses-*/anat/*.json")
l = [s.split("/")[1:3] for s in subjects_sessions]
assert len(set([ll[0] for ll in l])) == len(l), "Subjects have more than 1 visit each"
scans_params_df = []
for session in subjects_sessions:
    with open(session, 'r') as f:
        data = ujson.load(f)
        data["subject_id"] = session.split("/")[1].split("sub-")[-1]
        data["session_id"] = session.split("/")[2]
        scans_params_df.append(data)

scans_params_df = pd.DataFrame.from_records(scans_params_df)
scans_params_df

,Modality,MagneticFieldStrength,Manufacturer,ManufacturersModelName,DeviceSerialNumber,BodyPartExamined,PatientPosition,SoftwareVersions,MRAcquisitionType,SeriesDescription,...,DwellTime,ImageOrientationPatientDICOM,InPlanePhaseEncodingDirectionDICOM,ConversionSoftware,ConversionSoftwareVersion,subject_id,session_id,PhaseEncodingDirection,AcquisitionDateTime,RawImage
0,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[-0.013412, 0.996993, 0.076328, 0.0787298, 0.0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS30001,ses-M006,NaN,NaN,NaN
1,MR,1.5,Siemens,Sonata,21926,NaN,HFS,syngo_MR_2004A_4VA25A,3D,t1_mpr_ns_sag,...,NaN,"[-0.00523331, 0.999986, -2.92654e-08, -0.02966...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS30002,ses-M012,NaN,NaN,NaN
2,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[0.00264927, 0.994848, 0.101348, 0.0897104, 0....",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS30003,ses-M018,NaN,NaN,NaN
3,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[-0.03713, 0.997534, 0.0595549, 0.0199487, 0.0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS30004,ses-M036,NaN,NaN,NaN
4,MR,1.5,Siemens,Sonata,21926,NaN,HFS,syngo_MR_2004A_4VA25A,3D,t1_mpr_ns_sag,...,NaN,"[-0.177327, 0.984152, -4.2094e-08, -0.0648168,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS30005,ses-M006,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[-0.0398566, 0.998812, 0.0280487, -0.00500697,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS31168,ses-M006,NaN,NaN,NaN
1079,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[-0.00772917, 0.999578, -0.0279927, 0.0681862,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS31169,ses-M018,NaN,NaN,NaN
1080,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B17,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[0.0489637, 0.998076, 0.038044, -0.00828535, 0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS31170,ses-M078,NaN,NaN,NaN
1081,MR,3.0,Siemens,TrioTim,35177,HEAD,HFS,syngo_MR_B13_4VB13A,3D,t1_mpr_1mm_p2_pos50,...,0.000009,"[0.014959, 0.97729, 0.211375, -0.0200122, 0.21...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,OAS31171,ses-M024,NaN,NaN,NaN


In [9]:
scans_params_df.columns

Index(['Modality', 'MagneticFieldStrength', 'Manufacturer',
       'ManufacturersModelName', 'DeviceSerialNumber', 'BodyPartExamined',
       'PatientPosition', 'SoftwareVersions', 'MRAcquisitionType',
       'SeriesDescription', 'ScanningSequence', 'SequenceVariant',
       'ScanOptions', 'SequenceName', 'ImageType', 'AcquisitionTime',
       'AcquisitionNumber', 'SliceThickness', 'SAR', 'EchoTime',
       'RepetitionTime', 'InversionTime', 'FlipAngle', 'PartialFourier',
       'BaseResolution', 'ShimSetting', 'TxRefAmp', 'PhaseResolution',
       'PhaseOversampling', 'ReceiveCoilName', 'PulseSequenceDetails',
       'PercentPhaseFOV', 'PhaseEncodingSteps', 'AcquisitionMatrixPE',
       'ReconMatrixPE', 'ParallelReductionFactorInPlane', 'PixelBandwidth',
       'DwellTime', 'ImageOrientationPatientDICOM',
       'InPlanePhaseEncodingDirectionDICOM', 'ConversionSoftware',
       'ConversionSoftwareVersion', 'subject_id', 'session_id',
       'PhaseEncodingDirection', 'AcquisitionDateTi

In [10]:
scans_params_df.MagneticFieldStrength.value_counts()

3.000    898
1.500    168
1.494     17
Name: MagneticFieldStrength, dtype: int64

In [11]:
scans_params_df.Manufacturer.value_counts()

Siemens    1083
Name: Manufacturer, dtype: int64

In [12]:
scans_params_df.DeviceSerialNumber.value_counts()

35177    555
35248    190
51010    150
21926     33
26321      2
Name: DeviceSerialNumber, dtype: int64

In [13]:
scans_params_df.ManufacturersModelName.value_counts()

TrioTim         748
Biograph_mMR    150
Vision          150
Sonata           33
Avanto            2
Name: ManufacturersModelName, dtype: int64

In [14]:
scans_params_df.SequenceName.value_counts()

_tfl3d1_ns    800
tfl3d1_ns     130
Name: SequenceName, dtype: int64

In [15]:
scans_params_df.SequenceVariant.value_counts()

SP_MP_OSP    744
SP_MP        186
Name: SequenceVariant, dtype: int64

In [16]:
scans_params_df.ScanningSequence.value_counts()

GR_IR    897
IR_GR     33
Name: ScanningSequence, dtype: int64

In [18]:
scans_df = scans_sessions_df.merge(scans_params_df, on=["subject_id", "session_id"])
scans_df

,session_id,source_session_id,age,mmse,cdr,commun,dx1,homehobb,judgment,memory,...,ParallelReductionFactorInPlane,PixelBandwidth,DwellTime,ImageOrientationPatientDICOM,InPlanePhaseEncodingDirectionDICOM,ConversionSoftware,ConversionSoftwareVersion,PhaseEncodingDirection,AcquisitionDateTime,RawImage
0,ses-M006,d0129,65,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,...,2.0,220.0,0.000009,"[-0.013412, 0.996993, 0.076328, 0.0787298, 0.0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
1,ses-M012,d0371,68,28.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,NaN,130.0,NaN,"[-0.00523331, 0.999986, -2.92654e-08, -0.02966...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
2,ses-M018,d0558,60,29.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,2.0,220.0,0.000009,"[0.00264927, 0.994848, 0.101348, 0.0897104, 0....",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
3,ses-M036,d1101,58,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,2.0,220.0,0.000009,"[-0.03713, 0.997534, 0.0595549, 0.0199487, 0.0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
4,ses-M006,d0143,48,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,...,NaN,130.0,NaN,"[-0.177327, 0.984152, -4.2094e-08, -0.0648168,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,ses-M006,d0148,65,29.0,0.5,0.0,AD Dementia,0.5,0.5,0.5,...,2.0,220.0,0.000009,"[-0.0398566, 0.998812, 0.0280487, -0.00500697,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
1079,ses-M018,d0620,64,29.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,2.0,220.0,0.000009,"[-0.00772917, 0.999578, -0.0279927, 0.0681862,...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
1080,ses-M078,d2410,71,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,2.0,219.0,0.000009,"[0.0489637, 0.998076, 0.038044, -0.00828535, 0...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
1081,ses-M024,d0752,68,30.0,0.0,0.0,Cognitively normal,0.0,0.0,0.0,...,2.0,220.0,0.000009,"[0.014959, 0.97729, 0.211375, -0.0200122, 0.21...",ROW,dcm2niix,v1.0.20171017 GCC4.4.7,NaN,NaN,NaN
